# Tensorflow High Level APIs introduction

This notebook introduces the scala_tensorflow library high-level APIs (Dataset and Estimators) for a simple regression model case.

Summary:

- load data into a Tensor, make a Dataset
- Create a model Estimator
- Train
- Tensorflow training vizualization
- Save Model
- Model Signature
- Infer


In [ ]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("linux-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [ ]:
import sys.process._
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap

import java.nio.file.Paths
import scala.util.Random

In [ ]:
val rootDir = "resources/"  //sys.env("HOME") + "/data/immo/"

In [ ]:
s"head -3 ${rootDir}immo.csv"!

In [ ]:
val bufferedSource = scala.io.Source.fromFile(s"${rootDir}immo.csv")
lazy val tt = bufferedSource.getLines.drop(1).toVector
                       .map(_.split(",").map(_.trim.toFloat))
                       .map(arr => Tensor(arr(0), arr(1), arr(2), arr(3), arr(4)))

In [ ]:
val sess = Session()

In [ ]:
val dataTensor = Tensor(tt:_*)

val meanD = tf.mean(dataTensor, axes = Seq(0), keepDims = true, name = "Mean")
val varianceD = tf.mean( tf.squaredDifference(meanD, tf.stopGradient(dataTensor)), axes = Seq(0), keepDims = true, name = "Variance")

val dataScaled = tf.divide(tf.subtract(dataTensor, meanD), tf.sqrt(varianceD))
val dataScaledTensor = sess.run(fetches = dataScaled)

val trainFeatures = tf.data.datasetFromTensorSlices(dataScaledTensor(---, 1::))
val trainLabels   = tf.data.datasetFromTensorSlices(dataScaledTensor(---, 0))


In [ ]:
val trainData =
  trainFeatures.zip(trainLabels)
      .repeat()
      //.shuffle(10)
      .batch(250)
      .prefetch(1000)

In [ ]:
val input = Input(FLOAT32, Shape(-1, 4))
val trainInput = Input(FLOAT32, Shape(-1))

In [ ]:
import org.platanios.tensorflow.api.ops.variables._

In [ ]:
val layers =  Linear[Float]("Layer_0", 1)
val loss = L2Loss[Float, Float]("Loss") >> ScalarSummary(name = "Loss", tag = "Loss")

In [ ]:
val optimizer = tf.train.AdaGrad(1.0f)//tf.train.GradientDescent(1e-6f)
val model = Model.simpleSupervised(input, trainInput, layers, loss, optimizer)

In [ ]:
import org.platanios.tensorflow.api.learn.hooks._
import org.platanios.tensorflow.api.config.TensorBoardConfig


val summariesDir = java.nio.file.Paths.get("/tmp/001tfsc")

val estimator = InMemoryEstimator(
  modelFunction = model,
  configurationBase = Configuration(Some(summariesDir)),
  trainHooks = Set(
    SummarySaver(summariesDir, StepHookTrigger(10)),
    tf.learn.StepRateLogger(log = true, summaryDir = summariesDir, trigger = tf.learn.StepHookTrigger(100)),
    tf.learn.LossLogger(trigger = tf.learn.StepHookTrigger(10)),
    CheckpointSaver(summariesDir, StepHookTrigger(10))),
  tensorBoardConfig = TensorBoardConfig(summariesDir))

In [ ]:
estimator.train(() => trainData, StopCriteria(maxSteps = Some(1000)))

In [ ]:
val scaledPreds = estimator.infer(() => dataScaledTensor(---, 1::))
val predsOp = tf.add(meanD(---,0),tf.multiply(scaledPreds,tf.sqrt(varianceD(---,0))))
//val preds = tf.sum(meanD(---,0), tf.multiply(scaledPreds, tf.sqrt(varianceD(---,0))))
val (preds, m, sd2) = sess.run(fetches = (predsOp,meanD,varianceD))

In [ ]:
scaledPreds(---, 0).entriesIterator.toSeq

In [ ]:
sd2(---,0).entriesIterator.toSeq//.map(_)

In [ ]:
dataScaledTensor(---, 0).entriesIterator.toSeq.map(x =>x*x).reduce(_+_)